In [1]:
import numpy as np
import pandas as pd
import plotly.io as pio
import plotly.graph_objects as go

# settings
pio.templates.default = "simple_white"

In [2]:
data = pd.read_parquet('data_cleaned.parquet')
data

,Datum,Standort,Parameter,Wert,Lat,Lon
0,1983-01-01 00:00:00,Zch_Stampfenbachstrasse,CO in mg/m3,4.09,47.38239,8.54174
1,1983-01-01 00:00:00,Zch_Stampfenbachstrasse,NO2 in µg/m3,64.25,47.38239,8.54174
2,1983-01-01 00:00:00,Zch_Stampfenbachstrasse,NO in µg/m3,211.32,47.38239,8.54174
3,1983-01-01 00:00:00,Zch_Stampfenbachstrasse,NOx in ppb,203.02,47.38239,8.54174
4,1983-01-01 00:00:00,Zch_Stampfenbachstrasse,SO2 in µg/m3,126.44,47.38239,8.54174
...,...,...,...,...,...,...
210217,2021-12-31 23:00:00,Zch_Stampfenbachstrasse,SO2 in µg/m3,10.55,47.38239,8.54174
210216,2021-12-31 23:00:00,Zch_Stampfenbachstrasse,CO in mg/m3,0.54,47.38239,8.54174
210238,2021-12-31 23:00:00,Zch_Rosengartenstrasse,PM10 in µg/m3,61.90,47.39510,8.52592
210226,2021-12-31 23:00:00,Zch_Schimmelstrasse,NO2 in µg/m3,38.32,47.37027,8.52489


In [9]:
data_v1 = data[data["Parameter"] == "CO in mg/m3"][["Datum", "Parameter", "Wert"]].copy()
# get mean for every month
data_v1.loc[:, "Year"] = data_v1["Datum"].dt.year
data_v1.loc[:, "Month"] = data_v1["Datum"].dt.month

data_v1 = data_v1.groupby(["Year", "Month", "Parameter"]).mean().reset_index()
data_v1["Time"] = pd.to_datetime(data_v1[["Year", "Month"]].assign(DAY=1))
data_v1 = data_v1.drop(["Year", "Month", "Datum"], axis=1)

start_time = data_v1['Time'].min()
data_v1['time_years'] = data_v1['Time'].apply(lambda x: (x - start_time).days / 365.25)

# Map 'time_years' to a value between 0 and 2*pi for each year
data_v1['time_normalized'] = data_v1['time_years'].apply(lambda x: 2*np.pi*(x % 1))  # x%1 gives the fractional part

# Convert Time and Wert to 3D coordinates
data_v1['x'] = data_v1['Wert'] * np.cos(data_v1['time_normalized'])
data_v1['y'] = data_v1['Wert'] * np.sin(data_v1['time_normalized'])
data_v1['z'] = data_v1['time_years']

fig = go.Figure(data=[go.Scatter3d(
    x=data_v1['x'],
    y=data_v1['y'],
    z=data_v1['z'],
    mode='lines',
    line=dict(
        color=data_v1['Wert'],
        colorscale='Viridis',
        width=4
    ),
    name='',
    hovertemplate='%{text}',
    text="Date: " + data_v1['Time'].dt.strftime('%m.%Y') + '<br>Mean CO: ' + data_v1['Wert'].round(2).astype(str)
)])

fig.update_layout(
    scene=dict(
        xaxis_title="",
        yaxis_title="",
        zaxis_title='Year',
        camera=dict(
            up=dict(x=0, y=0, z=1),
            center=dict(x=0, y=0, z=0),
            eye=dict(x=1.3, y=1.3, z=0.5)
        ),
        aspectratio=dict(x=1, y=1, z=1)
    ),
    scene_xaxis_showgrid=False,
    scene_yaxis_showgrid=False,
    scene_zaxis_showgrid=False,
    scene_zaxis=dict(ticktext=['1983', '1993', '2003', '2013', '2023'], tickvals=[0, 10, 20, 30, 40]),
    scene_xaxis=dict(ticktext=[], tickvals=[]),
    scene_yaxis=dict(ticktext=[], tickvals=[]),
    scene_bgcolor='rgba(0,0,0,0)',
    template="simple_white",
    margin=dict(r=0, l=0, b=0, t=0)
)

fig.update_layout(
    title={
        'text': "Mean CO in mg/m3 in Zürich throughout the years",
        'y': 0.98,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    }
)

# display
fig.show()

In [4]:
# to html
fig.write_html("map.html")